In [1]:
import gensim
from gensim.models import Word2Vec
import re
import pymorphy2
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
# from my_parsers import download, parse
from tqdm import tqdm
from collections import Counter
# from brown_clustering import BigramCorpus, BrownClustering

pd.options.display.max_rows = 200
tqdm.pandas()

C:\Users\iltya\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\iltya\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\iltya\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\iltya\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_TRAIN_RES_1 = pd.read_csv('vprod_train/TRAIN_RES_1.csv')

#df_res = pd.concat([df_TRAIN_RES_1, df_TRAIN_RES_2, df_TRAIN_RES_3, df_TRAIN_RES_4, df_TRAIN_RES_5], axis=0)
# df_sal = pd.read_csv('vprod_train/TRAIN_SAL.csv')

In [3]:
# удаляем модифицированный столбец - он совпадает с точностью до регистра
df_TRAIN_RES_1.drop("achievements_modified", axis=1, inplace=True)

for text_feature in ["achievements", "company_name", "demands"]:
    df_TRAIN_RES_1[text_feature].fillna("", inplace=True)

df_TRAIN_RES_1["achievements_len"] = [len(i) for i in df_TRAIN_RES_1["achievements"]]
df_TRAIN_RES_1.drop_duplicates(inplace=True)

In [4]:
len(df_TRAIN_RES_1)

1664467

In [5]:
DROP_PART_DUPLICATES = False

if DROP_PART_DUPLICATES:
    df_TRAIN_RES_1["comp_name_demands"] = df_TRAIN_RES_1["demands"] + df_TRAIN_RES_1["company_name"]
    df_TRAIN_RES_1 = df_TRAIN_RES_1.drop_duplicates(subset="comp_name_demands")
    df_TRAIN_RES_1.drop("comp_name_demands", axis=1, inplace=True)

In [6]:
len(df_TRAIN_RES_1)

1664467

In [7]:
vc = df_TRAIN_RES_1["job_title"].value_counts().sort_values(ascending=False)
print("proportion of samples choised clases: ", vc.iloc[:200].values.sum() / vc.values.sum())

proportion of samples choised clases:  0.9580346140836676


In [8]:
used_classes = df_TRAIN_RES_1["job_title"].value_counts().sort_values(ascending=False).iloc[:200].index.tolist()
df_TRAIN_RES_1 = df_TRAIN_RES_1[df_TRAIN_RES_1["job_title"].isin(used_classes)]

In [9]:
len(df_TRAIN_RES_1)

1594617

In [10]:
df_TRAIN_RES_1 = df_TRAIN_RES_1.iloc[::5]

In [11]:
len(df_TRAIN_RES_1)

318924

In [24]:
df_TRAIN_RES_1["len_group"] = df_TRAIN_RES_1.groupby("id_cv").agg({"company_name": "count"})

In [28]:
clf_big_sphere = pickle.load(open("sphere_clf.pickle", "rb"))

C:\Users\iltya\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.4.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
# df_TRAIN_RES_1["id_cv_2"] = [i.split("-")[2] for i in df_TRAIN_RES_1["id_cv"]]
# df_TRAIN_RES_1["id_cv_4"] = [i.split("-")[4] for i in df_TRAIN_RES_1["id_cv"]]

## Word2vec common functions

In [29]:
# Инициализация анализатора pymorphy2 для лемматизации
morph = pymorphy2.MorphAnalyzer()

# Функция для лемматизации слова
def lemmatize_word(word):
    return morph.parse(word)[0].normal_form

# Функция для предобработки данных (разбиение на слова, лемматизация, удаление пунктуации)
def preprocess_names(company_names):
    processed_names = []
    for name in tqdm(company_names):
        # Приводим название к нижнему регистру
        name = name.lower()
        # Удаляем всю пунктуацию, оставляем только слова
        name = re.sub(r'[^\w\s]', '', name)
        # Разбиваем на отдельные слова
        words = name.split()

        # Лемматизируем каждое слово
        # lemmatized_words = [lemmatize_word(word) for word in words]

        processed_names.append(words)
    return processed_names


def get_emb_by_modele(model, comp_names_without_job, column_prefix):
    all_tokens = set(model.wv.index_to_key)
    word_vectors_dict = {word: model.wv[word] for word in model.wv.index_to_key}

    all_embds = []

    for sent in tqdm(comp_names_without_job):
        all_emb = [word_vectors_dict[word] for word in sent if word in all_tokens]

        if len(all_emb) == 0:
            emb = np.zeros(100)
        else:
            emb = np.mean(all_emb, axis=0)

        all_embds.append(emb)
    emb_comp_name_df = pd.DataFrame(np.array(all_embds), columns=[f"{column_prefix}_{i}" for i in range(100)])
    return emb_comp_name_df

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_emb_by_tfidf(processed_sentences, column_prefix, max_features=100, ):
    tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
    tfidf_matrix = tfidf_vectorizer.fit_transform(processed_sentences)

    # Преобразуем разреженную матрицу TF-IDF в плотный формат и создаем DataFrame
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=[f"{column_prefix}_{word}" for word in tfidf_vectorizer.get_feature_names_out()])
    return tfidf_vectorizer, tfidf_df

## w2v for company names

In [31]:
company_names = [i["company_name"] + " " + i["job_title"] for _, i in df_TRAIN_RES_1.iterrows()]
# Предобработка данных
sentences = preprocess_names(company_names)

100%|██████████| 318924/318924 [00:01<00:00, 226988.59it/s]


In [32]:
np.mean([len(i) for i in sentences])

4.5773287679823405

In [33]:
RETRAIN_MODEL = False

if RETRAIN_MODEL:
    # Обучение модели Word2Vec
    model_comp_name = Word2Vec(sentences, vector_size=64, window=4, min_count=1, sg=1)
    # Сохранение обученной модели
    model_comp_name.save("company_word2vec_russian.model")
else:
    model_comp_name = Word2Vec.load("company_word2vec_russian.model")

# model_comp_name.wv.most_similar('компьютер')

In [34]:
comp_names_without_job = preprocess_names(df_TRAIN_RES_1["company_name"])
emb_comp_name_df = get_emb_by_modele(model_comp_name, comp_names_without_job, column_prefix="comp_name_emb")
df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True), emb_comp_name_df.reset_index(drop=True)], axis=1)

100%|██████████| 318924/318924 [00:04<00:00, 71740.66it/s]


## w2v for demands

In [35]:
company_names = [i["demands"] + " " + i["job_title"] for _, i in df_TRAIN_RES_1.iterrows()]
# Предобработка данных
sentences = preprocess_names(company_names)

100%|██████████| 318924/318924 [00:03<00:00, 82325.49it/s] 


In [36]:
np.mean([len(i) for i in sentences])

33.00717412298855

In [37]:
RETRAIN_MODEL = False

if RETRAIN_MODEL:
    # Обучение модели Word2Vec
    model_demand = Word2Vec(sentences, vector_size=100, window=8, min_count=1, sg=1)
    # Сохранение обученной модели
    model_demand.save("demand_word2vec_russian.model")
else:
    model_demand = Word2Vec.load("demand_word2vec_russian.model")

model_demand.wv.most_similar('бухгалтер')

[('авансовые', 0.8037424683570862),
 ('кассыбанка', 0.7973775863647461),
 ('бух', 0.7930706739425659),
 ('кассабанк', 0.7926923036575317),
 ('фсспфр', 0.7875778079032898),
 ('налоговойстатистической', 0.7757202386856079),
 ('клинтбанк', 0.7753341197967529),
 ('кудир', 0.7724557518959045),
 ('заработная', 0.7707493305206299),
 ('пфрфссифнс', 0.7690602540969849)]

In [38]:
demand_without_job = preprocess_names(df_TRAIN_RES_1["demands"])
emb_demand_df = get_emb_by_modele(model_demand, demand_without_job, column_prefix="demand_emb")
df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True), emb_demand_df.reset_index(drop=True)], axis=1)

100%|██████████| 318924/318924 [00:11<00:00, 28783.95it/s]


In [39]:
df_TRAIN_RES_1.head()

,achievements,company_name,demands,id_cv,job_title,achievements_len,len_group,comp_name_emb_0,comp_name_emb_1,comp_name_emb_2,...,demand_emb_90,demand_emb_91,demand_emb_92,demand_emb_93,demand_emb_94,demand_emb_95,demand_emb_96,demand_emb_97,demand_emb_98,demand_emb_99
0,,Управление социальной защиты населения Сердобс...,"Работала,непосредственно, с федеральными льгот...",52860148-26da-11e8-a184-9122a281f90e,инспектор,0,NaN,-0.293772,-0.564505,-0.419635,...,0.183418,0.258860,0.290902,-0.200651,0.256295,0.253528,0.045275,-0.140245,-0.149471,-0.227559
1,,"ОАО «Высочайший» п.Бодайбо, Иркутская обл.",Разведка с применением поверхностных горных вы...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,0,NaN,-0.330126,-0.029948,0.391855,...,0.170837,0.391514,0.734838,0.099061,0.517577,0.271957,0.088179,-0.515044,-0.201592,-0.200995
2,,"ОАО «БАЙКАЛГЕО» п. Нижнеангарск, Бурятия",Проведение геологоразведочных работ на россыпн...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,заместитель,0,NaN,-0.318120,0.166147,-0.214572,...,0.150262,0.246992,0.499546,-0.141324,0.373341,0.260797,0.153022,-0.454782,-0.204571,-0.216284
3,"нет, т.к. участкам дано отрицательное заключение","ООО «Артель старателей «ТАЛ»», Усть-Нера, Респ...",руководил поисково-разведочными работами на бу...,d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,геолог,48,NaN,-0.201948,0.644514,-0.182316,...,0.123024,0.314913,0.608443,-0.120845,0.590910,0.153910,0.097952,-0.359757,-0.168976,-0.194691
4,,"Администрация Северо-Байкальского района, п.Ни...","Разработка программ развития района, статистич...",d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,заместитель,0,NaN,-0.586076,-0.180409,-0.088037,...,0.084129,0.042485,0.603362,-0.176048,0.255731,0.096839,0.305328,-0.284165,-0.173182,-0.192775


## add tf_idf company name

In [40]:
sentences = preprocess_names(df_TRAIN_RES_1["company_name"])

RETRAIN_MODEL = False

column_prefix = "comp_name_tfidf"

if RETRAIN_MODEL:
    comp_name_vectorizer, comp_name_tfidf = get_emb_by_tfidf([" ".join(sent) for sent in sentences],
                                                             column_prefix=column_prefix)
else:
    comp_name_vectorizer = pickle.load(open("comp_name_vectorizer.pkl", "rb"))

    comp_name_tfidf = comp_name_vectorizer.transform([" ".join(sent) for sent in sentences])
    comp_name_tfidf = pd.DataFrame(comp_name_tfidf.toarray(),
                                   columns=[f"{column_prefix}_{word}" for word in comp_name_vectorizer.get_feature_names_out()])

df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True),
                            comp_name_tfidf.reset_index(drop=True)], axis=1)

100%|██████████| 318924/318924 [00:01<00:00, 214620.03it/s]


In [41]:
sentences = preprocess_names(df_TRAIN_RES_1["demands"])

RETRAIN_MODEL = False
column_prefix = "demnds_tfidf"

if RETRAIN_MODEL:
    demands_vectorizer, demands_tfidf = get_emb_by_tfidf([" ".join(sent) for sent in sentences],
                                                     column_prefix=column_prefix)
else:
    demands_vectorizer = pickle.load(open("demands_vectorizer.pkl", "rb"))

    demands_tfidf = demands_vectorizer.transform([" ".join(sent) for sent in sentences])
    demands_tfidf = pd.DataFrame(demands_tfidf.toarray(),
                                 columns=[f"{column_prefix}_{word}" for word in demands_vectorizer.get_feature_names_out()])

df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True),
                            demands_tfidf.reset_index(drop=True)], axis=1)

100%|██████████| 318924/318924 [00:03<00:00, 95315.07it/s] 


In [42]:
sentences = preprocess_names(df_TRAIN_RES_1["achievements"])

RETRAIN_MODEL = False
column_prefix = "achiv_tfidf"

if RETRAIN_MODEL:
    achiv_vectorizer, achiv_tfidf = get_emb_by_tfidf([" ".join(sent) for sent in sentences],
                                                     column_prefix=column_prefix)
else:
    achiv_vectorizer = pickle.load(open("achiv_vectorizer.pkl", "rb"))
    achiv_tfidf = demands_vectorizer.transform([" ".join(sent) for sent in sentences])
    achiv_tfidf = pd.DataFrame(achiv_tfidf.toarray(),
                               columns=[f"{column_prefix}_{word}" for word in demands_vectorizer.get_feature_names_out()])

df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True),
                            achiv_tfidf.reset_index(drop=True)], axis=1)

100%|██████████| 318924/318924 [00:04<00:00, 65954.06it/s] 


In [43]:
df_TRAIN_RES_1.head()

,achievements,company_name,demands,id_cv,job_title,achievements_len,len_group,comp_name_emb_0,comp_name_emb_1,comp_name_emb_2,...,achiv_tfidf_техники,achiv_tfidf_товара,achiv_tfidf_товаров,achiv_tfidf_труда,achiv_tfidf_управление,achiv_tfidf_услуг,achiv_tfidf_участие,achiv_tfidf_учет,achiv_tfidf_учета,achiv_tfidf_формирование
0,,Управление социальной защиты населения Сердобс...,"Работала,непосредственно, с федеральными льгот...",52860148-26da-11e8-a184-9122a281f90e,инспектор,0,NaN,-0.293772,-0.564505,-0.419635,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,,"ОАО «Высочайший» п.Бодайбо, Иркутская обл.",Разведка с применением поверхностных горных вы...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,0,NaN,-0.330126,-0.029948,0.391855,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,,"ОАО «БАЙКАЛГЕО» п. Нижнеангарск, Бурятия",Проведение геологоразведочных работ на россыпн...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,заместитель,0,NaN,-0.318120,0.166147,-0.214572,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"нет, т.к. участкам дано отрицательное заключение","ООО «Артель старателей «ТАЛ»», Усть-Нера, Респ...",руководил поисково-разведочными работами на бу...,d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,геолог,48,NaN,-0.201948,0.644514,-0.182316,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,,"Администрация Северо-Байкальского района, п.Ни...","Разработка программ развития района, статистич...",d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,заместитель,0,NaN,-0.586076,-0.180409,-0.088037,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
list(df_TRAIN_RES_1.columns)

['achievements',
 'company_name',
 'demands',
 'id_cv',
 'job_title',
 'achievements_len',
 'len_group',
 'comp_name_emb_0',
 'comp_name_emb_1',
 'comp_name_emb_2',
 'comp_name_emb_3',
 'comp_name_emb_4',
 'comp_name_emb_5',
 'comp_name_emb_6',
 'comp_name_emb_7',
 'comp_name_emb_8',
 'comp_name_emb_9',
 'comp_name_emb_10',
 'comp_name_emb_11',
 'comp_name_emb_12',
 'comp_name_emb_13',
 'comp_name_emb_14',
 'comp_name_emb_15',
 'comp_name_emb_16',
 'comp_name_emb_17',
 'comp_name_emb_18',
 'comp_name_emb_19',
 'comp_name_emb_20',
 'comp_name_emb_21',
 'comp_name_emb_22',
 'comp_name_emb_23',
 'comp_name_emb_24',
 'comp_name_emb_25',
 'comp_name_emb_26',
 'comp_name_emb_27',
 'comp_name_emb_28',
 'comp_name_emb_29',
 'comp_name_emb_30',
 'comp_name_emb_31',
 'comp_name_emb_32',
 'comp_name_emb_33',
 'comp_name_emb_34',
 'comp_name_emb_35',
 'comp_name_emb_36',
 'comp_name_emb_37',
 'comp_name_emb_38',
 'comp_name_emb_39',
 'comp_name_emb_40',
 'comp_name_emb_41',
 'comp_name_emb_42',
 '

In [45]:
# with open("comp_name_vectorizer.pkl", "wb") as f:
#     pickle.dump(comp_name_vectorizer, f)
#
# with open("demands_vectorizer.pkl", "wb") as f:
#     pickle.dump(demands_vectorizer, f)
#
# with open("achiv_vectorizer.pkl", "wb") as f:
#     pickle.dump(achiv_vectorizer, f)



## add big sphere classifier predicts

In [53]:
emb_demand_df.columns

Index(['demand_emb_0', 'demand_emb_1', 'demand_emb_2', 'demand_emb_3',
       'demand_emb_4', 'demand_emb_5', 'demand_emb_6', 'demand_emb_7',
       'demand_emb_8', 'demand_emb_9', 'demand_emb_10', 'demand_emb_11',
       'demand_emb_12', 'demand_emb_13', 'demand_emb_14', 'demand_emb_15',
       'demand_emb_16', 'demand_emb_17', 'demand_emb_18', 'demand_emb_19',
       'demand_emb_20', 'demand_emb_21', 'demand_emb_22', 'demand_emb_23',
       'demand_emb_24', 'demand_emb_25', 'demand_emb_26', 'demand_emb_27',
       'demand_emb_28', 'demand_emb_29', 'demand_emb_30', 'demand_emb_31',
       'demand_emb_32', 'demand_emb_33', 'demand_emb_34', 'demand_emb_35',
       'demand_emb_36', 'demand_emb_37', 'demand_emb_38', 'demand_emb_39',
       'demand_emb_40', 'demand_emb_41', 'demand_emb_42', 'demand_emb_43',
       'demand_emb_44', 'demand_emb_45', 'demand_emb_46', 'demand_emb_47',
       'demand_emb_48', 'demand_emb_49', 'demand_emb_50', 'demand_emb_51',
       'demand_emb_52', 'demand_emb

In [54]:
emb_demand_df.columns = ["".join(i.split("_")[-2:]) for i in emb_demand_df.columns]

In [57]:
emb_demand_df.columns = [i[:3] + "_" + i[3:] for i in emb_demand_df.columns]

In [ ]:
# emb_demand_df.to_csv("emb_df.csv")

In [58]:
preds = []

for i in tqdm(range(0, len(emb_demand_df), 10000)):
    sl = emb_demand_df.iloc[i:i+10000]
    preds.extend(clf_big_sphere.predict(sl).tolist())

  0%|          | 0/32 [00:00<?, ?it/s]C:\Users\iltya\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
  3%|▎         | 1/32 [01:56<1:00:12, 116.52s/it]C:\Users\iltya\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic i

KeyboardInterrupt: 

## train model

In [46]:
len(df_TRAIN_RES_1)

318924

In [47]:
columns_to_drop = ["achievements", "company_name", "demands", "id_cv",]

In [31]:
X, y = df_TRAIN_RES_1.drop(columns_to_drop + ["job_title"], axis=1), df_TRAIN_RES_1["job_title"]

In [35]:
from sklearn.model_selection import train_test_split
N = 20_000
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:N], y.iloc[:N], stratify=y.iloc[:N], test_size=0.2, random_state=42)

In [36]:
from catboost import CatBoostClassifier, Pool

train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

In [38]:
clf = CatBoostClassifier(task_type="GPU", eval_metric="Accuracy")
clf.fit(train_pool, eval_set=test_pool, early_stopping_rounds=5)

Learning rate set to 0.139139
0:	learn: 0.2306875	test: 0.2202500	best: 0.2202500 (0)	total: 963ms	remaining: 16m 2s
1:	learn: 0.2551250	test: 0.2535000	best: 0.2535000 (1)	total: 1.81s	remaining: 15m 2s
2:	learn: 0.2706875	test: 0.2645000	best: 0.2645000 (2)	total: 2.69s	remaining: 14m 53s
3:	learn: 0.2812500	test: 0.2692500	best: 0.2692500 (3)	total: 3.55s	remaining: 14m 43s
4:	learn: 0.2923750	test: 0.2810000	best: 0.2810000 (4)	total: 4.44s	remaining: 14m 44s
5:	learn: 0.3030625	test: 0.2892500	best: 0.2892500 (5)	total: 5.32s	remaining: 14m 40s
6:	learn: 0.3121875	test: 0.2955000	best: 0.2955000 (6)	total: 6.2s	remaining: 14m 39s
7:	learn: 0.3197500	test: 0.3050000	best: 0.3050000 (7)	total: 6.99s	remaining: 14m 27s
8:	learn: 0.3243125	test: 0.3122500	best: 0.3122500 (8)	total: 7.78s	remaining: 14m 16s
9:	learn: 0.3355000	test: 0.3167500	best: 0.3167500 (9)	total: 8.67s	remaining: 14m 18s
10:	learn: 0.3439375	test: 0.3252500	best: 0.3252500 (10)	total: 9.57s	remaining: 14m 20s
11:

In [39]:
with open("clf.pkl", "wb") as f:
    pickle.dump(clf, f)

In [47]:
from collections import Counter
Counter([i[0] for i in clf.predict(X_test).tolist()])

Counter({'менеджер': 342,
         'электрогазосварщик': 14,
         'оператор': 223,
         'водитель': 130,
         'юрисконсульт': 91,
         'специалист': 686,
         'кладовщик': 38,
         'директор': 154,
         'мастер': 108,
         'администратор': 87,
         'кассир': 40,
         'слесарь': 93,
         'психолог': 14,
         'инженер': 444,
         'бухгалтер': 287,
         'руководитель': 42,
         'продавец': 94,
         'машинист': 67,
         'электромонтер': 73,
         'заместитель': 99,
         'экономист': 72,
         'фельдшер': 11,
         'консультант': 15,
         'инспектор': 54,
         'секретарь': 77,
         'архивариус': 1,
         'учитель': 77,
         'сборщик': 4,
         'механик': 34,
         'укладчик': 2,
         'медсестра': 7,
         'официант': 8,
         'монтажник': 37,
         'педагог': 14,
         'преподаватель': 76,
         'охранник': 23,
         'юрист': 22,
         'воспитатель': 42,
       

In [49]:
np.where(clf.predict(X_test) == "стрелок")

(array([3789], dtype=int64), array([0], dtype=int64))

In [63]:
df_TRAIN_RES_1[df_TRAIN_RES_1["comp_name_emb_0"] == X_test.iloc[3789]["comp_name_emb_0"]].iloc[0]["job_title"]

'стрелок'

In [57]:
X_test.iloc[3789]["comp_name_emb_0"]

0.4036295711994171